Train MLP for each finger pair!\
Using hyper param optimalization
Data source: \
sliding windowed powers for mu and beta band\
Subject 1

In [1]:
import os
import numpy as np
import h5py
from scipy import stats
import scipy.io
import mne
from random import shuffle
import random

mne.set_log_level('error')


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


import torch
import torch.nn as nn
import torch.optim as optim

import optuna


from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [2]:
# Set seed for NumPy
np.random.seed(42)

# Set seed for Python's built-in random number generator
random.seed(42)

# Set seed for PyTorch
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
subject_id = 0
tag = 'gpt4freq_all'

In [4]:
# Load the dictionary from the HDF5 file
target_dir = 'features'

file_path = os.path.join(target_dir, tag+'_'+cfg['subjects'][subject_id] + '.h5')


data = {}
with h5py.File(file_path, 'r') as h5file:
    for key in h5file.keys():
        data[key] = np.array(h5file[key])

# Print the loaded data dictionary
for key, value in data.items():
    data[key] = np.reshape(value, (value.shape[0], -1))
    print(key, data[key].shape)

index (50, 27540)
little (50, 27540)
middle (50, 27540)
ring (50, 27540)
thumb (50, 27540)


In [5]:
class SingleLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation):
        super(SingleLayerMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

In [12]:
def train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=100):
    device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device_name)
   
   
    model.to(device)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)


    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        y_pred = model(X_test)
        y_pred = torch.argmax(y_pred, dim=1)

    acc = accuracy_score(y_test.cpu(), y_pred.cpu())
    return acc

def objective(trial, X, y):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    num_epochs = trial.suggest_int("num_epochs", 100, 2000)
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    activation_name = trial.suggest_categorical("activation", ["relu", "elu", "leaky_relu"])
    optimizer = trial.suggest_categorical("optimizer", ["SGD", "Adam"])

    if activation_name == "relu":
        activation = nn.ReLU()
    elif activation_name == "elu":
        activation = nn.ELU()
    elif activation_name == "leaky_relu":
        activation = nn.LeakyReLU()

    if optimizer == "SGD":
        optimizer = optim.SGD
    elif optimizer == "Adam":
        optimizer = optim.Adam

    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
    model = SingleLayerMLP(train_X.shape[1], hidden_size, 2, activation)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    return train(train_X, train_y, test_X, test_y, model, criterion, optimizer, num_epochs=num_epochs)

    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    # fold_accuracies = []

    # for train_index, test_index in kf.split(X):
    #     X_train, X_test = X[train_index], X[test_index]
    #     y_train, y_test = y[train_index], y[test_index]

    #     model = SingleLayerMLP(X_train.shape[1], hidden_size, 2, activation)
    #     criterion = nn.CrossEntropyLoss()
    #     optimizer = optimizer(model.parameters(), lr=learning_rate)
    #     acc = train(X_train, y_train, X_test, y_test, model, criterion, optimizer, num_epochs=num_epochs)
    #     fold_accuracies.append(acc)

    # mean_accuracy = np.mean(fold_accuracies)
    # return mean_accuracy

def train_MLP(finger1, finger2, n_trials = 10, verbose = True):
   
    print(f'Training MLP for {finger1} vs {finger2}')

    X = np.concatenate((data[finger1], data[finger2]), axis=0)
    y = np.concatenate((np.ones(data[finger1].shape[0]), np.zeros(data[finger2].shape[0])), axis=0)

   
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=n_trials)

    best_trial = study.best_trial

    print(f'Best trial params: {best_trial.params}')
    print(f'Best trial accuracy: {best_trial.value * 100:.2f}%')
    return best_trial.value



In [13]:
train_MLP('middle', 'ring', n_trials = 1)

[I 2023-04-16 15:11:46,412] A new study created in memory with name: no-name-b50e14c2-f46d-4329-89a5-32728c4ec0f0


Training MLP for middle vs ring


[I 2023-04-16 15:11:48,686] Trial 0 finished with value: 0.85 and parameters: {'learning_rate': 0.0032452947897577667, 'num_epochs': 529, 'hidden_size': 52, 'activation': 'elu', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.85.


Best trial params: {'learning_rate': 0.0032452947897577667, 'num_epochs': 529, 'hidden_size': 52, 'activation': 'elu', 'optimizer': 'Adam'}
Best trial accuracy: 85.00%


0.85

In [14]:
accuracies = []
for finger1 in data:
    for finger2 in data:
        if finger1 != finger2:
            acc = train_MLP(finger1, finger2, n_trials = 100)
            accuracies.append(acc)
        else:
            break

print("#" * 50)
print(f'Mean accuracy: {np.mean(accuracies) * 100:.2f}%')
print(f'Std accuracy: {np.std(accuracies) * 100:.2f}%')
print("#" * 50)

[I 2023-04-16 15:11:53,660] A new study created in memory with name: no-name-aad5ed9e-cfd1-4b36-a535-296111d013b6


Training MLP for little vs index


[I 2023-04-16 15:11:58,341] Trial 0 finished with value: 0.45 and parameters: {'learning_rate': 4.225341346784311e-05, 'num_epochs': 1674, 'hidden_size': 23, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.45.
[I 2023-04-16 15:11:59,888] Trial 1 finished with value: 0.4 and parameters: {'learning_rate': 0.00015058196570379848, 'num_epochs': 1112, 'hidden_size': 76, 'activation': 'leaky_relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.45.
[I 2023-04-16 15:12:01,985] Trial 2 finished with value: 0.65 and parameters: {'learning_rate': 0.0005450982301314723, 'num_epochs': 1366, 'hidden_size': 36, 'activation': 'leaky_relu', 'optimizer': 'Adam'}. Best is trial 2 with value: 0.65.
[I 2023-04-16 15:12:02,936] Trial 3 finished with value: 0.5 and parameters: {'learning_rate': 0.05626028212078556, 'num_epochs': 797, 'hidden_size': 85, 'activation': 'elu', 'optimizer': 'SGD'}. Best is trial 2 with value: 0.65.
[I 2023-04-16 15:12:03,801] Trial 4 finished with val

In [ ]:
##################################################
# Mean accuracy: 77.50%
# Std accuracy: 11.88%
##################################################